In [ ]:
# Mount Google Drive to access files (if needed)
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import nltk
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

# Check if GPU is available and use it
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) == 0:
    print("No GPU available. Using CPU instead.")
else:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print(f'GPU {physical_devices[0]} available: True')

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Define a function for text preprocessing (stemming and cleaning)
def preprocess_text(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.split()
    ps = PorterStemmer()
    text = [ps.stem(word) for word in text if not word in set(stopwords.words('english'))]
    text = ' '.join(text)
    return text

# Load dataset (adjust path as per your file location)
news_dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dataset/fakeNewsData.csv')
news_dataset = news_dataset.fillna('')

# Combine author and title into content
news_dataset['content'] = news_dataset['author'] + ' ' + news_dataset['title']
news_dataset['content'] = news_dataset['content'].apply(preprocess_text)

# Split dataset into train and test sets
X = news_dataset['content']
y = news_dataset['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# Vectorization using TF-IDF
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train).toarray()
X_test_tfidf = tfidf.transform(X_test).toarray()

# Define simple feed-forward neural network model
input_layer = Input(shape=(X_train_tfidf.shape[1],))  # Shape for TF-IDF input
dense_layer_1 = Dense(128, activation='relu')(input_layer)
dropout_layer = Dropout(0.5)(dense_layer_1)
dense_layer_2 = Dense(64, activation='relu')(dropout_layer)
output_layer = Dense(1, activation='sigmoid')(dense_layer_2)

model = Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

print("\nTraining model with TF-IDF Features...")
history = model.fit(X_train_tfidf, y_train, epochs=10, batch_size=32, validation_data=(X_test_tfidf, y_test), verbose=1)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
No GPU available. Using CPU instead.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



Training model with TF-IDF Features...
Epoch 1/10
455/455 [==============================] - 9s 17ms/step - loss: 0.1824 - accuracy: 0.9398 - val_loss: 0.0497 - val_accuracy: 0.9827
Epoch 2/10
455/455 [==============================] - 6s 12ms/step - loss: 0.0217 - accuracy: 0.9933 - val_loss: 0.0413 - val_accuracy: 0.9845
Epoch 3/10
455/455 [==============================] - 6s 14ms/step - loss: 0.0061 - accuracy: 0.9987 - val_loss: 0.0443 - val_accuracy: 0.9859
Epoch 4/10
455/455 [==============================] - 6s 14ms/step - loss: 0.0025 - accuracy: 0.9995 - val_loss: 0.0460 - val_accuracy: 0.9857
Epoch 5/10
455/455 [==============================] - 5s 11ms/step - loss: 0.0011 - accuracy: 0.9998 - val_loss: 0.0504 - val_accuracy: 0.9862
Epoch 6/10
455/455 [==============================] - 7s 16ms/step - loss: 4.3781e-04 - accuracy: 1.0000 - val_loss: 0.0565 - val_accuracy: 0.9843
Epoch 7/10
455/455 [==============================] - 5s 12ms/step - loss: 2.8500e-04 - accuracy: 

In [ ]:

from sklearn.metrics import confusion_matrix, roc_auc_score, precision_score, recall_score, f1_score, matthews_corrcoef

# Accuracy
loss_attention, accuracy_attention = model.evaluate(X_test_tfidf, y_test)

print(f"\nTest Accuracy: {accuracy_attention}")
print(f"\nLoss: {loss_attention}")

# Predict probabilities for test set
y_pred_prob = model.predict(X_test_tfidf)

# Convert probabilities to binary predictions (0 or 1)
y_pred = (y_pred_prob > 0.5).astype(int)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print(f"Confusion Matrix:\n{cm}")

# AUC Score
auc_score = roc_auc_score(y_test, y_pred_prob)
print(f"AUC Score: {auc_score:.4f}")

# Calculate True Positives, True Negatives, False Positives, False Negatives
tn, fp, fn, tp = cm.ravel()

# Sensitivity (Recall)
sensitivity = tp / (tp + fn)
print(f"Sensitivity (Recall): {sensitivity:.4f}")

# Specificity
specificity = tn / (tn + fp)
print(f"Specificity: {specificity:.4f}")

# Precision
precision = precision_score(y_test, y_pred)
print(f"Precision: {precision:.4f}")

# F1 Score
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.4f}")

# Matthews Correlation Coefficient (MCC)
mcc = matthews_corrcoef(y_test, y_pred)
print(f"Matthews Correlation Coefficient (MCC): {mcc:.4f}")

195/195 [==============================] - 1s 3ms/step - loss: 0.0641 - accuracy: 0.9849

Test Accuracy: 0.9849358797073364

Loss: 0.0641297698020935
195/195 [==============================] - 1s 3ms/step
Confusion Matrix:
[[3109   39]
 [  55 3037]]
AUC Score: 0.9987
Sensitivity (Recall): 0.9822
Specificity: 0.9876
Precision: 0.9873
F1 Score: 0.9848
Matthews Correlation Coefficient (MCC): 0.9699
